# Data Cleaning:


In [1]:
## Overviewing the dataset ##

#top 5 rows
data.head()


## MISSING VALUES ##

# count
missing_values_count = data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

# missing percentage
total_cells = np.product(data.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells) * 100

# Drop missing values
data.dropna() #only rows
data.dropna(axis=1) #only columns

# Replace missing values
data.fillna(0) #replace with zeros
data.fillna(method='bfill', axis=0).fillna(0) #replace with value that comes directly after it in the same column, zero to remaning



## SCALING & NORMALIZATION ##

# mix-max scale the data between 0 and 1
from mlxtend.preprocessing import minmax_scaling
data = minmax_scaling(original_data, columns=[0])

# plot with minmax and without minmax scaling
import seaborn as sns
import matplotlib.pyplot as plt
original = np.random.exponential(size=100)
scaled = minmax_scaling(original_data, columns=[0])
fig, ax = plt.subplots(1,2)
sns.distplot(original_data, ax=ax[0])
ax[0].set_title("Original")
sns.distplot(scaled_data, ax=ax[1])
ax[1].set_title("Scaled")

# normalize with boxcox
from scipy import stats
normalized_data = stats.boxcox(data)

# plot with and without normalization
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax=plt.subplots(1,2)
sns.distplot(original_data, ax=ax[0])
ax[0].set_title("Original Data")
sns.distplot(normalized_data[0], ax=ax[1])
ax[1].set_title("Normalized data")



## DATE PARSING ##

# convert date columns to datetime
import datetime
data['date_parsed'] = pd.to_datetime(data['date'], format="%m/%d/%y")

# Some examples:
# 1/17/07 has the format "%m/%d/%y"
# 17-1-2007 has the format "%d-%m-%Y"

# to get day of the month
day_of_month = data['date_parsed'].dt.day
day_of_month.head()

# plot day of the month
sns.distplot(day_of_month, kde=False, bins=31)



## CHARACTER ENCODING ##

#UTF-8 is the standard text encoding. 
#All Python code is in UTF-8 and, ideally, all your data should be as well. 
#It's when things aren't in UTF-8 that you run into trouble.

# check to see what datatype it is
type(data)

# testing
before = "This is the euro symbol: €"
type(before)
#OUTPUT: str
after = before.encode("utf-8", errors="replace")
type(after)
#OUTPUT: byte

after = before.encode("ascii", errors="replace")
print(after.decode("ascii"))

# to check encoding type
with open("../path", 'rb') as rawdata:
    char_encoding = chardet.detect(rawdata.read(10000))
print(char_encoding)

# reading file using result encoding type
data = pd.read_csv("../path.csv", encoding='Windows-1252') # Windows-1252 came from checking encoding



## INCONSISTENT DATA ENTRY ##

# Fuzzy matching #
# get the top 10 closest matches to "lolo"
data = fuzzywuzzy.process.extract("lolo", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
# take a look at them
data

# function to replace rows in the provided column of the provided dataframe
# ratios to be taken from fuzzy results i.e. data
def replace_matches_in_column(df, column, string_to_match, min_ratio = 50):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

     # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("Done")


NameError: ignored